# Этот ноутбук позволяет запустить своего телеграм-бота для стилизации изображений в анимешный стиль. Для стилизации используется нейросеть [AnimeGAN](https://github.com/TachibanaYoshino/AnimeGAN)

Для запуска необходимо: 

1.   В верхнем меню выбрать File -> Save a copy in Drive (откроется новая вкладка, все дальнейшие действия нужно выполнять в ней)
2.   В верхнем меню выбрать Runtime -> change runtime type -> GPU

3.   В телеграме найти аккаунт @BotFather и с помощью команды /newbot создать нового бота.
4.   BotFather после создания бота даст токен -- набор букв цифр, их необходимо скопировать и вставить в форму ниже (через Ctrl + V).
5.   В верхнем меню выбрать Runtime -> Run All

После этих действий можно отправлять созданному боту фотографии.
Бот будет работать пока открыт браузер с этой страницей. Огрнаничения гул-колаба не позволяют выполнять код (а следовательно и работать боту) более 12 часов. 




In [0]:
#@title Введите ваш токен
token = "" #@param {type:"string"}


In [0]:
#@title Устанавливаем необходимые библиотеки
from IPython.display import clear_output
!pip install pytelegrambotapi
%tensorflow_version 1.x
!git clone  https://github.com/TachibanaYoshino/AnimeGAN.git
%cd AnimeGAN
!wget https://github.com/TachibanaYoshino/AnimeGAN/releases/download/Haoyao-style_V1.0/Haoyao-style.zip
!unzip /content/AnimeGAN/Haoyao-style.zip -d pppoint
!rm /content/AnimeGAN/Haoyao-style.zip
%cd /content/AnimeGAN
!mkdir img
clear_output()

In [0]:
#@title Запускаем бота!
import test
import telebot
import os
import tensorflow as tf
bot = telebot.TeleBot(token, threaded=False)



@bot.message_handler(commands=['start', 'help'])
def handle_start_help(message):
	bot.send_message(message.chat.id, 'Этот бот стилизует изображения в анимешный стиль с помощью нейросети AnimeGAN. Просто отправьте боту фото.\nИ да, подписывайтесь на канал @NeuralShit')


@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):

    try:

        tf.reset_default_graph () 
        file_info = bot.get_file(message.photo[len(message.photo)-1].file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        cut = file_info.file_path.split('/')
        cut = cut[1]

        src='/content/AnimeGAN/img/'+ cut
        with open(src, 'wb') as new_file:
           new_file.write(downloaded_file)
        bot.reply_to(message,"Фото добавлено в очередь на обработку")
        test.test('/content/AnimeGAN/pppoint', 'H', '/content/AnimeGAN/img/')


        with open ('/content/AnimeGAN/results/H/'+cut , 'rb' ) as snd:
          animeme = snd.read()
        bot.send_photo(message.chat.id, animeme)
        os.remove('/content/AnimeGAN/results/H/' + cut)
        os.remove('/content/AnimeGAN/img/' + cut)
        clear_output()


    except Exception as e:
        print (e)


@bot.message_handler(content_types=['document'])
def handle_docs_photo(message):
  bot.reply_to(message, 'Отправьте файл как "фото", а не как "документ" ')


bot.infinity_polling(none_stop=True)

 
